
References:

* [AWS IoT Python](https://github.com/aws/aws-iot-device-sdk-python/blob/master/README.rst)
* [AWS CA](https://docs.aws.amazon.com/en_pv/iot/latest/developerguide/server-authentication.html)


prepares:

```python
pip install AWSIoTPythonSDK kipp3
```

In [29]:
import logging
import os
import time
import json
from concurrent.futures import ThreadPoolExecutor
from threading import Event

from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient
from kipp3.utils import setup_logger
from kipp3.decorator import debug_wrapper


logger = setup_logger("awsiot")
executor = ThreadPoolExecutor(max_workers=10)

In [2]:
# Configure logging
logging.getLogger("AWSIoTPythonSDK.core").setLevel(logging.INFO)


# streamHandler = logging.StreamHandler()
# formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
# streamHandler.setFormatter(formatter)
# logger.addHandler(streamHandler)

In [43]:
# config_path = "/Users/laisky/repo/laisky/configs"
config_path = "/opt/configs"

cli = AWSIoTMQTTClient("laisky-demo")
cli.configureEndpoint("a2mth8rkdw0p8j-ats.iot.us-west-2.amazonaws.com", 8883)
cli.configureCredentials(
    os.path.join(config_path, "mqtt/root-CA.crt"),
    os.path.join(config_path, "mqtt/python-cli.private.key"),
    os.path.join(config_path, "mqtt/python-cli.cert.pem"),
)

cli.configureAutoReconnectBackoffTime(1, 32, 20)
cli.configureOfflinePublishQueueing(-1)  # Infinite offline Publish queueing
cli.configureDrainingFrequency(2)  # Draining: 2 Hz
cli.configureConnectDisconnectTimeout(10)  # 10 sec
cli.configureMQTTOperationTimeout(10)  # 5 sec


def customCallback(cli, userdata, message):
    print("Received a new message: ")
    print(message.payload)
    print("from topic: ")
    print(message.topic)
    print("--------------\n\n")


cli.connect()


True

[2019-12-17 06:22:02,617 - INFO - <ipython-input-42-3c9882da16d9>:8 - awsiot] - >> topic='laisky-hello' publish True, payload='{"yo": "jedi"}'
[2019-12-17 06:22:02,960 - INFO - <ipython-input-41-3c9882da16d9>:8 - awsiot] - >> topic='laisky-hello' publish True, payload='{"yo": "jedi"}'


In [5]:
stop_evt = Event()

In [46]:
TOPIC = "laisky-hello"


@debug_wrapper
def publisher(topic:str, stop_evt: Event):
    while not stop_evt.is_set():
        payload = json.dumps({"yo": "jedi"})
        logger.info(f">> {topic=} publish {cli.publish(topic, payload, 1)}, {payload=}")
        time.sleep(3)

    logger.info("publisher stop")


@debug_wrapper
def stop_all_publisher():
    stop_evt.set()

    
def get_subscriber(topic):
    logger.info(f"subscribe to {topic=}")
    @debug_wrapper
    def callback(client, userdata, msg):
        logger.info(
            f"<< {topic=} got: {userdata=}, {msg.mid=}, {msg.state=}, {msg.retain=}, {msg.topic=}, {msg.payload=}"
        )
        
    return callback

topic = TOPIC
# cli.unsubscribe(TOPIC)

cli.subscribe(topic, 1, get_subscriber(topic))
fp = executor.submit(publisher, topic, stop_evt)


[2019-12-17 06:22:15,548 - INFO - <ipython-input-46-3c9882da16d9>:20 - awsiot] - subscribe to topic='laisky-hello'
[2019-12-17 06:22:15,952 - INFO - <ipython-input-46-3c9882da16d9>:8 - awsiot] - >> topic='laisky-hello' publish True, payload='{"yo": "jedi"}'
[2019-12-17 06:22:15,972 - INFO - <ipython-input-46-3c9882da16d9>:23 - awsiot] - << topic='laisky-hello' got: userdata=None, msg.mid=1, msg.state=0, msg.retain=0, msg.topic='laisky-hello', msg.payload=b'{"yo": "jedi"}'
[2019-12-17 06:22:15,973 - INFO - /Users/laisky/repo/laisky/kipp/kipp3/decorator/__init__.py:129 - kipp] - callback cost 0.00s
[2019-12-17 06:22:18,956 - INFO - <ipython-input-46-3c9882da16d9>:8 - awsiot] - >> topic='laisky-hello' publish False, payload='{"yo": "jedi"}'
[2019-12-17 06:22:21,235 - INFO - <ipython-input-46-3c9882da16d9>:23 - awsiot] - << topic='laisky-hello' got: userdata=None, msg.mid=1, msg.state=0, msg.retain=0, msg.topic='laisky-hello', msg.payload=b'{"yo": "jedi"}'
[2019-12-17 06:22:21,237 - INFO -

In [47]:
stop_evt.set()
stop_evt = Event()

[2019-12-17 06:22:28,376 - INFO - <ipython-input-46-3c9882da16d9>:11 - awsiot] - publisher stop
[2019-12-17 06:22:28,378 - INFO - /Users/laisky/repo/laisky/kipp/kipp3/decorator/__init__.py:129 - kipp] - publisher cost 12.63s
